# Imports

In [141]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
from pprint import pprint

# Load and Split Data

In [142]:
# Read in the data
df = pd.read_csv('data/train.csv')
df_len = len(df)
df.head()

,TransactionID,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,...,C6,C7,C8,C9,C10,C11,C12,C13,C14,isFraud
0,1,C,5812,408.0,185.0,mastercard,224.0,debit,NotFound,NotFound,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0
1,2,W,12577,268.0,150.0,visa,166.0,debit,123.0,87.0,...,94.0,0.0,0.0,85.0,0.0,104.0,0.0,457.0,120.0,0
2,3,H,12581,453.0,150.0,visa,226.0,debit,272.0,87.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0
3,4,W,8139,327.0,150.0,discover,194.0,credit,204.0,87.0,...,2.0,0.0,0.0,2.0,0.0,3.0,0.0,18.0,6.0,0
4,5,W,16132,111.0,150.0,visa,226.0,debit,220.0,87.0,...,2.0,0.0,0.0,1.0,0.0,1.0,0.0,28.0,1.0,0


In [143]:
training_data, testing_data = train_test_split(df, test_size=0.20, random_state=100, shuffle=True)
#training_data.head()

In [144]:
#testing_data.head()
label_column = training_data

# Is our data pure?

In [145]:
def check_purity(data):
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)
    
    if len(unique_classes) == 2:
        return True
    else: 
        return False

In [146]:
check_purity(training_data[training_data.card1 < 5000].values) 

True

# Classify

In [147]:
# def classify_data(data):
#     
#     return classification